# Задача предсказания победителя в онлайн-игре DOTA 2

## Проверка гипотезы о влиянии синергии персонажей на качество модели

Евгений Колонский, ekolonsky@gmail.com

При постановке и решении задачи предсказания победителя в игре DOTA 2 часто принимается как факт, что состав команд (пять персонажей команды светлых против пяти персонажей команды темных) существенно влияет на результат игры. (см., например, работы [1], [2], [3]). Считается, что мощь удачно подобранной команды больше суммы сил отдельных персонажей. Этот эффект называется синергией. На гипотезе синергии строятся алгоритмы, предсказывающие результат игры до ее начала, на основании только состава команд светлых и темных.


[1] http://cs229.stanford.edu/proj2013/PerryConley-HowDoesHeSawMeARecommendationEngineForPickingHeroesInDota2.pdf

[2] http://cseweb.ucsd.edu/~jmcauley/cse255/reports/wi15/Kaushik_Kalyanaraman.pdf

[3] http://cseweb.ucsd.edu/~jmcauley/cse255/reports/fa15/018.pdf
    

Попробуем проверить гипотезу о синергии. Если она верна, то чем сильнее совпадение состава команд в двух независимых играх, тем больше шансов совпадения результата игры.

In [1]:
import pandas as pd
import numpy as np 

### Загрузка данных

In [2]:
data = pd.read_csv('features.csv', index_col='match_id')
n_samples = data.shape[0]


Y = data['radiant_win']

# y = array of +1 when r win and -1 when d win
y = map(lambda x: -1 if x==0 else 1, Y.values)
y = np.array(y)

### Схожесть, расстояние

Степень совпадения состава ("схожесть") команд в матчах $i$ и $j$ измеряется целым числом от 0 до 10. Схожесть равна 10 при полном совпадении команд (пять светлых и пять темных игры $i$ совпали с составом команд игры $j$), и нулю при полном несовпадении. Понятие схожести здесь обратно понятию расстояния: чем ближе матчи друг к другу по составу команд, тем больше схожесть и меньше "расстояние".


Воспользуемся тем, что количество возможных персонажей $N=113$ меньше 128, количества бит в длинном целом.
Будем кодировать состав команды единичными битами в 128-битном целом числе.

In [3]:
# rbit, dbit - массивы, кодирующие составы команд Radiant и Dire.
rbit = [0L] * (n_samples)
dbit = [0L] * (n_samples)

for i, match_id in enumerate(data.index):
    for player_ind in ['1','2','3','4','5']:

        hero_ind = int(data.ix[match_id, 'r'+player_ind+'_hero'])
        rbit[i] += 1L << hero_ind

        hero_ind = int(data.ix[match_id, 'd'+player_ind+'_hero'])
        dbit[i] += 1L << hero_ind


In [4]:
def bitCount(int_type):
    """ подсчет количества единиц в двоичном числе"""
    count = 0
    while(int_type):
        int_type &= int_type - 1
        count += 1
    return(count)

def similarity(i,j):
    """ схожесть команд в i и j игре:
        возвращает целое число от 0 (нет совпадений)
        до 10 (полное совпадение).
        второй ответ для зеркального случая
        когда когда в матче $j$ команды меняются местами."""
    if i == j:
        return 0, 0
    # прямое сравнение составов команд
    simA = bitCount(rbit[i] & rbit[j]) + bitCount(dbit[i] & dbit[j])
    # зеркало - в j матче светлые и темные поменялись друг с другом
    simB = bitCount(rbit[i] & dbit[j]) + bitCount(dbit[i] & rbit[j])    
    return simA, simB


## Идея проверки

Идея проверки в следующем.

Переберем все пары матчей $i=1..n$ и $j=1..n$ обучающей выборки.  $n=97230$ - объем обучающей выборки, 
Для каждой пары вычислим схожесть $similarity(i,j)=0..10$  и совпадение результата игры $y_i = y_j$.

Будем накапливать счетчики: сколько пар матчей имеют определенную схожесть, и у скольки из них совпали результаты. Таким образом, для каждого уровня схожести $l=0..10$ будет получена  метрика выброчной вероятности совпадения результатов игры $p_l$ как отношение счетчика совпадений результатов к счетчику пар.

Для более полной утилизации результатов будем считать, что для каждого матча существует "зеркальный" ему матч, в котором команды светлых и темных поменялись местами, а  результат зеркального совпадает с точностью до знака с результатом основного матча.

In [ ]:
# Этот расчет идет несколько часов, лучше оставить на его считать на ночь..
# .. или перейти к следуюшей ячейке и загрузить готовый результат

counter = [0L] * 11 # счетчики пар матчей
confirm = [0L] * 11 # счетчики совпадения результатов пары матчей

for i in xrange(n_samples):

    for j in xrange(i+1, n_samples):
        levelA, levelB = similarity(i,j)
        counter[levelA] += 1
        if y[i] == y[j]:
            confirm[levelA] += 1
        counter[levelB] += 1
        if y[i] == -y[j]:
                confirm[levelB] += 1

    if i % 100 == 0:
        print 'Processed %d' %i
        print 'counter ', counter
        print 'confirm ', confirm 

np.save('counter', counter)
np.save('confirm', confirm)

In [12]:
# .. загрузить результат 
counter = np.load('counter.npy')
confirm = np.load('confirm.npy')

In [13]:
print np.log10(sum(counter)) # примерно 10^10 записей. Точно n*(n-1)

9.97559610481


In [14]:
print counter
print confirm

[3592498889 3562654304 1676459310  499164010  104770310   16085070
    1795592     140764       7161        257          3]
[1791789474 1781469259  840516904  250999809   52868221    8153288
     914626      72443       3680        129          2]


In [15]:
# Доверительные интервалы для среднего
from statsmodels.stats.proportion import proportion_confint

In [16]:
for i in range(11):
    print proportion_confint(confirm[i], counter[i], method='wilson'), counter[i], i


(0.4987421825076917, 0.4987748825586959) 3592498889 0
(0.50002346954063803, 0.50005630637233067) 3562654304 1
(0.50134039888208237, 0.50138826739823306) 1676459310 2
(0.50279649489494338, 0.50288421910160019) 499164010 3
(0.50451497729768024, 0.50470645160308614) 104770310 4
(0.50664112818092877, 0.50712977534014014) 16085070 5
(0.50864173108622135, 0.51010413545168121) 1795592 6
(0.51203028910384796, 0.51725196710809973) 140764 7
(0.50231422622374056, 0.52546028925489696) 7161 8
(0.44123953149933781, 0.56259421488638317) 257 9
(0.20765960080204771, 0.93850805527960368) 3 10


In [18]:
# Доля пар матчей со степнью совпадения 2 и больше
print float(sum(counter[2:])) / sum(counter)

0.243127315762


In [20]:
# средний эффект от синергии уровня 2 и выше
print float(sum(confirm[2:])) / sum(counter[2:]) - 0.5

0.00187862046391


# Выводы

### 1.  Синергия уровня 2 и больше встречается в ~25% случаев

### 2. Средний эффект от синергии уровня 2 и выше ~0.2%

### 3. Верхняя граница довер. интервала синергии уровня 8: ~ 2.5%

## Влияние синергии на результат незначительно